# Regression

This notebook estimates the area of squares and circles, in pixels. 

In [1]:
import os
import torch
import torch.nn as nn
from torch import optim

from cnn_framework.dummy_regression_cnn.data_set import DummyRegressionCnnDataSet
from cnn_framework.dummy_regression_cnn.model_params import DummyModelParams
from cnn_framework.dummy_regression_cnn.model import DummyCnn

from cnn_framework.utils.data_loader_generators.data_loader_generator import DataLoaderGenerator
from cnn_framework.utils.model_managers.regression_model_manager import RegressionModelManager
from cnn_framework.utils.data_managers.default_data_manager import DefaultDataManager
from cnn_framework.utils.metrics import MeanErrorMetric
from cnn_framework.utils.create_dummy_data_set import generate_data_set

In [2]:
params = DummyModelParams()
params.update()

# Create data set if needed
if not os.path.exists(params.data_dir):
    generate_data_set(params.data_dir)
    print(f"\nData set created in {params.data_dir}")

print(f"\nModel will be saved in {params.models_folder}")
print(f"Predictions will be saved in {params.output_dir}")
print(f"Tensorboard logs will be saved in {params.tensorboard_folder_path}")

Model time id: 20230908-154454-local
epochs 50 | batch 2 | lr 0.001 | weight decay 0.0 | dropout 0.0

Model will be saved in C:\Users\thoma\models/local/dummy_regression_cnn/20230908-154454-local
Predictions will be saved in C:\Users\thoma\predictions/local/dummy_regression_cnn/20230908-154454-local
Tensorboard logs will be saved in C:\Users\thoma\tensorboard/local/20230908-154454-local_dummy_regression_cnn


In [3]:
loader_generator = DataLoaderGenerator(params, DummyRegressionCnnDataSet, DefaultDataManager)
train_dl, val_dl, test_dl = loader_generator.generate_data_loader()

### Data source ###
train data is loaded from C:\Users\thoma\data\dummy - 80% elements
val data is loaded from C:\Users\thoma\data\dummy - 10% elements
test data is loaded from C:\Users\thoma\data\dummy - 10% elements
###################
train has 160 images.
val has 20 images.
test has 20 images.
###################


In [4]:
model = DummyCnn(
    nb_classes=params.nb_classes,
    nb_input_channels=params.nb_modalities * params.nb_stacks_per_modality,
)

manager = RegressionModelManager(model, params, MeanErrorMetric)

optimizer = optim.Adam(
    model.parameters(), lr=float(params.learning_rate), betas=(params.beta1, params.beta2),
)  # define the optimization

loss_function = nn.L1Loss()  # define the loss function

Current commit hash: 589eca1e521ba7aa867d3d42d738cdbe2fda362f


In [5]:
manager.fit(train_dl, val_dl, optimizer, loss_function)

Training in progress: 100.0% | Local step 80 | Epoch 50
Best model saved at epoch 40.

Training successfully finished in 0:01:20.475006.


In [6]:
for model_path, name in zip(
    [manager.model_save_path_early_stopping, manager.model_save_path],
    ["early stopping", "final"],
):
    print(f"\nPredicting with {name} model.")
    # Update model with saved one
    manager.model.load_state_dict(torch.load(model_path))
    manager.predict(test_dl)


Predicting with early stopping model.
Model evaluation in progress: 100.0% | Batch #9
Average MeanError: -137.99

Predicting with final model.
Model evaluation in progress: 100.0% | Batch #9
Average MeanError: -261.53
